In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from tqdm import tqdm
import logging
import math
import os
import random
import sys
import time
import pickle
import copy

import numpy as np
import torch

from fairseq import (
    checkpoint_utils,
    distributed_utils,
    options,
    quantization_utils,
    tasks,
    utils,
)
from fairseq.data import iterators
from fairseq.logging import meters, metrics, progress_bar
from fairseq.trainer import Trainer
from fairseq.model_parallel.megatron_trainer import MegatronTrainer
from fairseq.models.pruned_transformer import PrunedTransformerModel

In [3]:
args = pickle.load(open("argsfile.p", "rb"))

In [4]:
task = tasks.setup_task(args)

task.load_dataset(args.valid_subset, combine=False, epoch=1)
dataset = task.dataset(args.valid_subset)

checkpoint_dir = "/home/raj/data/raj-learn/checkpoints/lr-rewind_0.75sparsity_0.2frac_30epochs/"
model_paths = ["checkpoint_LTH0_epoch60.pt",
              "checkpoint_LTH1_epoch60_sparsity0.168.pt",
              "checkpoint_LTH2_epoch60_sparsity0.302.pt",
              "checkpoint_LTH3_epoch60_sparsity0.410.pt", 
              "checkpoint_LTH4_epoch60_sparsity0.496.pt", 
              "checkpoint_LTH5_epoch60_sparsity0.565.pt",
              "checkpoint_LTH6_epoch60_sparsity0.620.pt",
              "checkpoint_LTH7_epoch60_sparsity0.664.pt"]



In [5]:
args.path = checkpoint_dir + model_paths[0]

In [6]:
models, _model_args = checkpoint_utils.load_model_ensemble(
        args.path.split(os.pathsep),
        task=task,
    )
model = models[0]
model.cuda()
model.eval()

PrunedTransformerModel(
  (encoder): TransformerEncoder(
    (embed_tokens): Embedding(32768, 1024, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): 

In [7]:
itr = task.get_batch_iterator(
            dataset=dataset,
            max_tokens=args.max_tokens,
            max_sentences=args.max_sentences,
            max_positions=utils.resolve_max_positions(
                task.max_positions(),
                *[m.max_positions() for m in models],
            ),
            ignore_invalid_inputs=args.skip_invalid_size_inputs_valid_test,
            required_batch_size_multiple=args.required_batch_size_multiple,
            seed=args.seed,
            num_shards=args.distributed_world_size,
            shard_id=args.distributed_rank,
            num_workers=args.num_workers,
        ).next_epoch_itr(shuffle=False)

In [8]:
all_hidden = {}
for batch in tqdm(itr):
    ids = batch["id"].cpu().numpy().tolist()
    src_lens = batch["net_input"]["src_lengths"].cpu().numpy()
    enc_outputs = model.encoder(batch["net_input"]["src_tokens"].cuda(), batch["net_input"]["src_lengths"].cuda(), 
                                        return_all_hiddens=True, return_all_attns=False)
#     print(enc_outputs.encoder_states[0].shape)
    enc_states = [x.detach().cpu().numpy() for x in enc_outputs.encoder_states]
    del enc_outputs
    
    out, props = model(batch["net_input"]["src_tokens"].cuda(), batch["net_input"]["src_lengths"].cuda(), \
                        batch["net_input"]["prev_output_tokens"].cuda())
    dec_states = [x.detach().cpu().numpy() for x in props["inner_states"]]
#     print(props["inner_states"][0].shape)
#     print(batch["net_input"]["prev_output_tokens"][0,:])
    del out, props
    

    for i, id_ in enumerate(ids):
        hiddens = {"enc_hidden":[], "dec_hidden":[]}
        tgt_seq = batch["net_input"]["prev_output_tokens"][i,:].numpy()
        tgt_len = tgt_seq.shape[0] - np.count_nonzero(tgt_seq == 1)

        for j in range(6):
            hiddens["enc_hidden"].append(enc_states[j][:src_lens[i],i,:])
            hiddens["dec_hidden"].append(dec_states[j][:tgt_len,i,:])
        all_hidden[id_] = hiddens
        
    del batch
    
    break

  0%|          | 0/34 [00:00<?, ?it/s]
